# Image Processing Summary
Author: Melissa

This NB uses the file archive_image_list.txt, which was generated for all images from both 2021A-0113 and 2021B-0149 in the NOIRLab Data Archive.

**Class:** MLG has assigned all archive images a "class", as follows.<br>
Some images were obtained with a different exposure time and are not "standard".<br>
We don't want to bother with those or with any other exposures.

`| Image Class  | filters | exposure times |     |`<br>
`| COSMOS       | g r i   | 60  86  130    | std |`<br>
`| COSMOS-MOON  | g r i   | 60  30  30     |     |`<br>
`| COSMOS-AGN   | g r i   | 80  70  90     |     |`<br>
`| ELAIS        | g r i   | 60  86  130    | std |`<br>
`| DECaPS-East  | g r i z | 96  50  30  30 | std |`<br>
`| DECaPS-West  | g r i z | 96  50  30  30 | std |`<br>

## Connect to database

In [ ]:
import psycopg2
import psycopg2.extras
import getpass

import os
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from astropy.coordinates import SkyCoord

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
tables = ['exposures','subtractions']
for table in tables:
    query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
    cursor.execute( query, ( table, ))
    print( f"\nTABLE: {table}\n===========================" )
    for row in cursor:
        print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

## NOIRLab Archive Image List

Melissa made this list of all 2021A-0113 and 2021B-0149 images from the NOIRLab Data Archive.

* fbase : base of archive filename (without extension)
* ra, dec : coordinates in decimal degrees
* mjd : modified julian date of observation (at exposure start)
* filt : filter
* airm : airmass
* obj : object name
* expt : exposure time
* seqid : sequence id (relates to JSON file for observing)
* **class : COSMOS, COSMOS-AGN, COSMOS-MOON, ELAIS, DECaPS-East, DECaPS-West, or NONE**
   * -AGN and -MOON indicate non-standard exposure times
   * class = COSMOS, ELAIS, or DECaPS are considered "standard"
* ftype : field type, EGAL, GAL, or NONE
* moonsep : moon separation in degrees
* moonill : moon illumination as a fraction
* mpval : 0.2 quantile sky background value for central chip (for images with moonsep<60 and moonill>0.2)

Recall that mpval used to be just the median image pixel value for images with moonsep<60 and moonill>0.2.

In [ ]:
fnm = 'archive_image_list.txt'
arch_fbase   = np.loadtxt( fnm, dtype='str', usecols=(0) )
arch_ra      = np.loadtxt( fnm, dtype='float', usecols=(1) )
arch_dec     = np.loadtxt( fnm, dtype='float', usecols=(2) )
arch_mjd     = np.loadtxt( fnm, dtype='float', usecols=(3) )
arch_filt    = np.loadtxt( fnm, dtype='str', usecols=(4) )
arch_airm    = np.loadtxt( fnm, dtype='float', usecols=(5) )
arch_obj     = np.loadtxt( fnm, dtype='str', usecols=(6) )
arch_expt    = np.loadtxt( fnm, dtype='float', usecols=(7) )
arch_seqid   = np.loadtxt( fnm, dtype='str', usecols=(8) )
arch_class   = np.loadtxt( fnm, dtype='str', usecols=(9) )
arch_ftype   = np.loadtxt( fnm, dtype='str', usecols=(10) )
arch_moonsep = np.loadtxt( fnm, dtype='float', usecols=(11) )
arch_moonill = np.loadtxt( fnm, dtype='float', usecols=(12) )
arch_mpval   = np.loadtxt( fnm, dtype='float', usecols=(13) )

#### get rid of the DECaPS and non-standard archive images

In [ ]:
tx = np.where( (arch_class != 'COSMOS') & (arch_class != 'ELAIS') )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(arch_class))

arch_fbase   = np.delete( arch_fbase, tx )
arch_ra      = np.delete( arch_ra, tx )
arch_dec     = np.delete( arch_dec, tx )
arch_mjd     = np.delete( arch_mjd, tx )
arch_filt    = np.delete( arch_filt, tx )
arch_airm    = np.delete( arch_airm, tx )
arch_obj     = np.delete( arch_obj, tx )
arch_expt    = np.delete( arch_expt, tx )
arch_seqid   = np.delete( arch_seqid, tx )
arch_class   = np.delete( arch_class, tx )
arch_ftype   = np.delete( arch_ftype, tx )
arch_moonsep = np.delete( arch_moonsep, tx )
arch_moonill = np.delete( arch_moonill, tx )
arch_mpval   = np.delete( arch_mpval, tx )
del tx

print('new length of arch arrays = ',len(arch_fbase))

## Exposures and subtractions tables

### exposures

In [ ]:
query = "SELECT id, ra, dec, filename, mjd, filter, proposalid, header FROM exposures"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
exp_id  = np.asarray( results[0], dtype='int' )
exp_ra  = np.asarray( results[1], dtype='float' )
exp_dec = np.asarray( results[2], dtype='float' )
exp_fnm = np.asarray( results[3], dtype='str' )
exp_mjd = np.asarray( results[4], dtype='float' )
exp_fil = np.asarray( results[5], dtype='str' )
exp_pid = np.asarray( results[6], dtype='str' )
exp_hdr = np.asarray( results[7], dtype='str' )
del query, results

In [ ]:
### exp_fbs : just the filename base (without the extension)
### exp_cal : calendar date
exp_fbs = deepcopy(exp_fnm)
exp_cal = np.zeros( len(exp_fnm), dtype='int' )

for i,fnm in enumerate(exp_fnm):
    tmp1 = fnm.split('.')[0]
    exp_fbs[i] = tmp1
    tmp2 = tmp1.split('_')[1]
    exp_cal[i] = int(tmp2)
    del tmp1,tmp2

# print('fnm: ', exp_fnm[0])
# print('fbs: ',exp_fbs[0])
# print('cal: ',exp_cal[0])

In [ ]:
### The header is a string that can be converted to a directory.
# temp = eval(exp_hdr[0])
# temp
# print("temp['AZ'] = ",temp['AZ'])

#### show map of all exposures

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot( exp_ra, exp_dec, 'o', ms=10, alpha=0.01, mew=0, color='grey')
plt.plot( 150.0 ,  +3.1,     '*', ms=6, alpha=1, color='dodgerblue')
plt.plot( 149.22,  +1.75,    '*', ms=6, alpha=1, color='dodgerblue')
plt.plot( 150.78,  +1.75,    '*', ms=6, alpha=1, color='dodgerblue')
plt.plot( 7.8746,  -43.0096, '*', ms=6, alpha=1, color='darkviolet')
plt.plot( 9.5   ,  -43.9980, '*', ms=6, alpha=1, color='darkviolet')
plt.plot( 270.89,  -29.53,   '*', ms=6, alpha=1, color='darkorange')
plt.plot( 116.32,  -26.25,   '*', ms=6, alpha=1, color='yellowgreen')
print('there are no DECaPS fields in the exposures table')

#### get rid of non-DDF and non-standard exposures

In [ ]:
### exp_aid : archive identifier (index in archive array)
exp_aid = np.zeros( len(exp_id), dtype='int' ) - 1

tag = 0
for i,fbase in enumerate(arch_fbase):
    tx = np.where( fbase == exp_fbs )[0]
    if len(tx) == 1:
        exp_aid[tx[0]] = i
    elif len(tx) > 1:
        exp_aid[tx[0]] = i
        tag += 1
    del tx

print(tag,' cases of double-matches, only the first will be kept')
del tag

In [ ]:
tx = np.where( exp_aid == -1 )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(exp_aid))

exp_id = np.delete( exp_id, tx )
exp_ra = np.delete( exp_ra, tx )
exp_dec = np.delete( exp_dec, tx )
exp_fnm = np.delete( exp_fnm, tx )
exp_mjd = np.delete( exp_mjd, tx )
exp_fil = np.delete( exp_fil, tx )
exp_pid = np.delete( exp_pid, tx )
exp_hdr = np.delete( exp_hdr, tx )
exp_fbs = np.delete( exp_fbs, tx )
exp_cal = np.delete( exp_cal, tx )
exp_aid = np.delete( exp_aid, tx )
del tx

print('new length of exp arrays = ',len(exp_id))

In [ ]:
# plt.figure(figsize=(12, 4))
# plt.plot( exp_ra, exp_dec, 'o', ms=10, alpha=0.01, mew=0, color='grey')
# plt.plot( 150.0 ,  +3.1,     '*', ms=6, alpha=1, color='dodgerblue')
# plt.plot( 149.22,  +1.75,    '*', ms=6, alpha=1, color='dodgerblue')
# plt.plot( 150.78,  +1.75,    '*', ms=6, alpha=1, color='dodgerblue')
# plt.plot( 7.8746,  -43.0096, '*', ms=6, alpha=1, color='darkviolet')
# plt.plot( 9.5   ,  -43.9980, '*', ms=6, alpha=1, color='darkviolet')
# plt.plot( 270.89,  -29.53,   '*', ms=6, alpha=1, color='darkorange')
# plt.plot( 116.32,  -26.25,   '*', ms=6, alpha=1, color='yellowgreen')

### subtractions

In [ ]:
query = "SELECT id, ra, dec, lmt_mg, seeing, skysig, magzp, "+ \
        "ccdnum, image_id, exposure_id FROM subtractions"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
sub_id  = np.asarray( results[0], dtype='int' )
sub_ra  = np.asarray( results[1], dtype='float' )
sub_dec = np.asarray( results[2], dtype='float' )
sub_lmg = np.asarray( results[3], dtype='float' )
sub_see = np.asarray( results[4], dtype='float' )
sub_sks = np.asarray( results[5], dtype='float' )
sub_mzp = np.asarray( results[6], dtype='float' )
sub_ccd = np.asarray( results[7], dtype='int' )
sub_iid = np.asarray( results[8], dtype='int' )
sub_eid = np.asarray( results[9], dtype='int' )
del query, results

#### get rid of subtractions that aren't matched to exposure table

In [ ]:
### sub_eid : exposure identifier (index in exposure array)
sub_tmp = np.zeros( len(sub_id), dtype='int' ) - 1

tag = 0
for i in range(len(exp_id)):
    tx = np.where( exp_id[i] == sub_eid )[0]
    if len(tx) > 0:
        sub_tmp[tx] = i
    else:
        tag += 1
    del tx

print(tag,' exposures had no subtractions')
del tag

In [ ]:
tx = np.where( sub_tmp == -1 )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(sub_id))
sub_id  = np.delete(sub_id, tx )
sub_ra  = np.delete(sub_ra, tx )
sub_dec = np.delete(sub_dec, tx )
sub_lmg = np.delete(sub_lmg, tx )
sub_see = np.delete(sub_see, tx )
sub_sks = np.delete(sub_sks, tx )
sub_mzp = np.delete(sub_mzp, tx )
sub_ccd = np.delete(sub_ccd, tx )
sub_iid = np.delete(sub_iid, tx )
sub_eid = np.delete(sub_eid, tx )

print('new length of sub arrays = ',len(sub_id))
del sub_tmp,tx

### add qualities to the sub and exp tables

In [ ]:
### sub_fil : the filter of the exposure associated with the subtraction
### sub_mjd : the MJD of the exposure associated with the subtraction
### sub_era : the exposure ra
### sub_ede : the exposure dec
### sub_off : the offset in arcsec of the sub ccd from the image center

sub_fil = np.zeros( len(sub_eid), dtype='str' )
sub_mjd = np.zeros( len(sub_eid), dtype='float' )
sub_era = np.zeros( len(sub_eid), dtype='float' )
sub_ede = np.zeros( len(sub_eid), dtype='float' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    if len(tx) == 1:
        sub_fil[i] = exp_fil[tx[0]]
        sub_mjd[i] = exp_mjd[tx[0]]
        sub_era[i] = exp_ra[tx[0]]
        sub_ede[i] = exp_dec[tx[0]]
    elif len(tx) == 0:
        print('No exposure id match for: ', sub_eid[i])
    elif len(tx) > 1:
        print('More than one exposure id match for: ', sub_eid[i])
    del tx

c1 = SkyCoord(sub_ra,sub_dec,unit="deg")
c2 = SkyCoord(sub_era,sub_ede,unit="deg")
sub_off = c1.separation(c2).arcsec

del sub_era,sub_ede,c1,c2

In [ ]:
### exp_mdn_see : median seeing over all subtractions for this exposure
### exp_mdn_lmg : median limiting magnitude over all subtractions for this exposure
### exp_mdn_sks : median skysig over all subtractions for this exposure
### exp_mdn_mzp : median zeropoint over all subtractions for this exposure

exp_mdn_see = np.zeros( len(exp_id), dtype='float' )
exp_mdn_lmg = np.zeros( len(exp_id), dtype='float' )
exp_mdn_sks = np.zeros( len(exp_id), dtype='float' )
exp_mdn_mzp = np.zeros( len(exp_id), dtype='float' )

tag = 0
for i in range(len(exp_id)):
    sx = np.where( sub_eid == exp_id[i] )[0]
    if len(sx) > 0:
        exp_mdn_see[i] = np.median(sub_see[sx])
        exp_mdn_lmg[i] = np.median(sub_lmg[sx])
        exp_mdn_sks[i] = np.median(sub_sks[sx])
        exp_mdn_mzp[i] = np.median(sub_mzp[sx])
    elif len(sx) == 0:
        tag += 1
    del sx
print('number of exposures with no subtractions: ',tag,' out of ',len(exp_id),' exposures')
del tag

In [ ]:
### sub_rel_see : subtraction seeing / median seeing for all subs of exp
### sub_rel_lmg : " but for limiting magnitude
### sub_rel_sks : " but for skysig
### sub_rel_mzp : " but for zeropoint

sub_rel_see = np.zeros( len(sub_eid), dtype='float' )
sub_rel_lmg = np.zeros( len(sub_eid), dtype='float' )
sub_rel_sks = np.zeros( len(sub_eid), dtype='float' )
sub_rel_mzp = np.zeros( len(sub_eid), dtype='float' )

for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    if len(tx) == 1:
        sub_rel_see[i] = sub_see[i] / exp_mdn_see[tx[0]]
        sub_rel_lmg[i] = sub_lmg[i] / exp_mdn_lmg[tx[0]]
        sub_rel_sks[i] = sub_sks[i] / exp_mdn_sks[tx[0]]
        sub_rel_mzp[i] = sub_mzp[i] / exp_mdn_mzp[tx[0]]
    del tx

In [ ]:
### sub_expt    : exposure time of the image for this subtraction
### sub_moonsep : moon separation in degrees of the image for this subtraction
### sub_moonill : moon illumination (fraction) of the image for this subtraction
### sub_aid     : archive image list id for this subtraction
### sub_cls     : archive image class for this subtraction

sub_expt    = np.zeros( len(sub_eid), dtype='float' )
sub_moonsep = np.zeros( len(sub_eid), dtype='float' )
sub_moonill = np.zeros( len(sub_eid), dtype='float' )
sub_mpixval = np.zeros( len(sub_eid), dtype='float' )

for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    if len(tx) == 1:
        archid         = exp_aid[tx[0]]
        sub_expt[i]    = arch_expt[archid]
        sub_moonsep[i] = arch_moonsep[archid]
        sub_moonill[i] = arch_moonill[archid]
        sub_mpixval[i] = arch_mpval[archid]

In [ ]:
### Print value ranges for the exposure id, subtraction id, subtraction ccd
# print(np.min(exp_id),np.max(exp_id))
# print(np.min(sub_ccd),np.max(sub_ccd))
# print(np.min(sub_iid),np.max(sub_iid))
# print(np.min(sub_eid),np.max(sub_eid))

In [ ]:
### Are there any bad ccds?
### Yes. CCD = 31, 61 have no subtractions
### Known issue: http://www.ctio.noao.edu/noao/node/2630

# usebins = np.arange( 63, dtype='float' ) + 0.5
# plt.hist(sub_ccd, bins=usebins, histtype='step')
# plt.xlabel('CCD Number')
# plt.ylabel('Number of Subtractions')

### Print number of subtractions per CCD as a check
# for i in range(63):
#     tx = np.where( sub_ccd == i )[0]
#     print(i, len(tx))
#     del tx

### show subtractions map

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot( sub_ra, sub_dec, 'o', ms=10, alpha=0.01, color='grey')

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
cx = np.where( (sub_ra>145) & (sub_ra<155) & (sub_dec>0.0) & (sub_dec<8.0) )[0]
plt.plot( sub_ra[cx], sub_dec[cx], 'o', ms=10, alpha=0.01, color='dodgerblue')
plt.subplot(1, 2, 2)
ex = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_dec>-48.0) & (sub_dec<-40.0) )[0]
plt.plot( sub_ra[ex], sub_dec[ex], 'o', ms=10, alpha=0.01, color='darkviolet')

#### show it's only 'standard' exposure times

In [ ]:
# plt.figure(figsize=(12,4))
# plt.hist(sub_expt)

for e in np.unique( np.sort( sub_expt ) ):
    tx = np.where( sub_expt == e )[0]
    print( e, len(tx) )
    del tx

<br>
<br>

## Plot histograms of image characteristics

In [ ]:
### COSMOS subtractions in g,r,i
cxg = np.where( (sub_ra>145) & (sub_ra<155) & (sub_fil == 'g') )[0]
cxr = np.where( (sub_ra>145) & (sub_ra<155) & (sub_fil == 'r') )[0]
cxi = np.where( (sub_ra>145) & (sub_ra<155) & (sub_fil == 'i') )[0]

### ELAIS subtractions in g,r,i
exg = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'g') )[0]
exr = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'r') )[0]
exi = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'i') )[0]

clr_cg = 'darkgreen'
clr_eg = 'limegreen'
clr_cr = 'firebrick'
clr_er = 'orange'
clr_ci = 'saddlebrown'
clr_ei = 'lightcoral'

In [ ]:
### seeing

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_see[cxg],sub_see[exg]], bins=30, stacked=True, color=[clr_cg,clr_eg], label=['COSMOS','ELAIS'])
plt.xlabel('Seeing [arcsec], g-band')
plt.xlim([0,3])
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper right')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_see[cxr],sub_see[exr]], bins=30, stacked=True, color=[clr_cr,clr_er], label=['COSMOS','ELAIS'])
plt.xlabel('Seeing [arcsec], r-band')
plt.xlim([0,3])
plt.legend(loc='upper right')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_see[cxi],sub_see[exi]], bins=30, stacked=True, color=[clr_ci,clr_ei], label=['COSMOS','ELAIS'])
plt.xlabel('Seeing [arcsec], i-band')
plt.xlim([0.5,3])
plt.legend(loc='upper right')
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/imgproc_seeing_dist')
plt.show()

### sky background

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [np.log10(sub_sks[cxg]),np.log10(sub_sks[exg])], bins=30, stacked=True, color=[clr_cg,clr_eg], label=['COSMOS','ELAIS'])
plt.xlabel('log(sigma_sky) [log(counts)], g-band')
plt.ylabel('Number of Processed CCDs')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [np.log10(sub_sks[cxr]),np.log10(sub_sks[exr])], bins=30, stacked=True, color=[clr_cr,clr_er], label=['COSMOS','ELAIS'])
plt.xlabel('log(sigma_sky) [log(counts)], r-band')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [np.log10(sub_sks[cxi]),np.log10(sub_sks[exi])], bins=30, stacked=True, color=[clr_ci,clr_ei], label=['COSMOS','ELAIS'])
plt.xlabel('log(sigma_sky) [log(counts)], i-band')
plt.xlim([0.25,2])
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/imgproc_logsky_dist')
plt.show()

### limiting magnitude

plt.figure(figsize=(12, 4))
plt.rcParams.update({'font.size': 13})
# plt.subplots(1, 3, sharey=True)
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_lmg[cxg],sub_lmg[exg]], bins=30, stacked=True, color=[clr_cg,clr_eg], label=['COSMOS','ELAIS'])
plt.xlabel('Limiting Magnitude, g-band')
plt.ylabel('Number of Processed CCDs')
plt.xlim([21,24.5])
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_lmg[cxr],sub_lmg[exr]], bins=30, stacked=True, color=[clr_cr,clr_er], label=['COSMOS','ELAIS'])
plt.xlabel('Limiting Magnitude, r-band')
plt.xlim([21,24.5])
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_lmg[cxi],sub_lmg[exi]], bins=30, stacked=True, color=[clr_ci,clr_ei], label=['COSMOS','ELAIS'])
plt.xlabel('Limiting Magnitude, i-band')
plt.xlim([21,24.5])
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/imgproc_limmag_dist')
plt.show()

### zeropoint

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_mzp[cxg],sub_mzp[exg]], bins=50, stacked=True, color=[clr_cg,clr_eg], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (g)')
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_mzp[cxr],sub_mzp[exr]], bins=50, stacked=True, color=[clr_cr,clr_er], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (r)')
plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_mzp[cxi],sub_mzp[exi]], bins=50, stacked=True, color=[clr_ci,clr_ei], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (i)')
plt.legend(loc='upper left')
plt.xlim([28,30.5])
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/imgproc_zeropoint_dist')
plt.show()

<br>
<br>

## Plot image quality parameters over time

In [ ]:
### seeing vs. MJD

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_see[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_mjd[exg], sub_see[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_see[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_mjd[exr], sub_see[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('r-band MJD')
# plt.ylabel('r-band seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], sub_see[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_mjd[exi], sub_see[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band seeing')
plt.show()

### sky background vs. MJD

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], np.log10(sub_sks[cxg]), 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_mjd[exg], np.log10(sub_sks[exg]), 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('MJD')
plt.ylabel('Log(sigma_sky)')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], np.log10(sub_sks[cxr]), 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_mjd[exr], np.log10(sub_sks[exr]), 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('MJD')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], np.log10(sub_sks[cxi]), 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_mjd[exi], np.log10(sub_sks[exi]), 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('MJD')
plt.show()

### limiting magnitude vs. MJD

plt.figure(figsize=(12, 3))

ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_mjd[exg], sub_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('MJD')
plt.ylabel('Limiting Magnitude')
# plt.legend(loc='upper left')

ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_mjd[cxr], sub_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_mjd[exr], sub_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('MJD')
# plt.legend(loc='upper left')

ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_mjd[cxi], sub_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_mjd[exi], sub_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('MJD')
# plt.legend(loc='upper left')

plt.show()

### zeropoint vs. MJD

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_mjd[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_mjd[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('r-band MJD')
# plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], sub_mzp[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_mjd[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band mag_zp')
plt.show()

<br>
<br>

## Image quality parameter correlations

In [ ]:
ymin = 21.0
ymax = 24.8
### limiting magnitude vs. seeing

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_see[cxg], sub_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_see[exg], sub_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('Seeing [arcsec], g-band')
plt.ylabel('Limiting Magnitude [mag]')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_see[cxr], sub_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_see[exr], sub_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('Seeing [arcsec], r-band')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_see[cxi], sub_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_see[exi], sub_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('Seeing [arcsec], i-band')
plt.xlim([0.5,3.0])
plt.ylim([ymin,ymax])
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/imgproc_seeing_vs_limmag')
plt.show()

### limiting magnitude vs. sky background

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( np.log10(sub_sks[cxg]), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg)
plt.plot( np.log10(sub_sks[exg]), sub_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg)
plt.xlabel('log(sigma_sky) [log(counts)], g-band')
plt.ylabel('Limiting Magnitude [mag]')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( np.log10(sub_sks[cxr]), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr)
plt.plot( np.log10(sub_sks[exr]), sub_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er)
plt.xlabel('log(sigma_sky) [log(counts)], r-band')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( np.log10(sub_sks[cxi]), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci)
plt.plot( np.log10(sub_sks[exi]), sub_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei)
plt.xlabel('log(sigma_sky) [log(counts)], i-band')
plt.xlim([0.35,2.0])
plt.ylim([ymin,ymax])
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/imgproc_skysig_vs_limmag')
plt.show()

### limiting magnitude vs. moon separation

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_moonsep[cxg]+(2.5*np.random.rand(len(cxg))), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
plt.plot( sub_moonsep[exg]+(2.5*np.random.rand(len(exg))), sub_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
plt.xlabel('Moon Separation [deg]')
plt.ylabel('Limiting Magnitude')
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_moonsep[cxr]+(2.5*np.random.rand(len(cxr))), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
plt.plot( sub_moonsep[exr]+(2.5*np.random.rand(len(exr))), sub_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
plt.xlabel('Moon Separation [deg]')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_moonsep[cxi]+(2.5*np.random.rand(len(cxi))), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
plt.plot( sub_moonsep[exi]+(2.5*np.random.rand(len(exi))), sub_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
plt.xlabel('Moon Separation [deg]')
# plt.ylabel('i-band lmt_mg')
plt.ylim([ymin,ymax])
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/imgproc_moonsep_vs_limmag')
plt.show()

### limiting magnitude vs. moon illumination

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_moonill[cxg]+(0.025*np.random.rand(len(cxg))), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg)
plt.plot( sub_moonill[exg]+(0.025*np.random.rand(len(exg))), sub_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg)
plt.xlabel('Moon Illumination Fraction')
plt.ylabel('Limiting Magnitude')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_cg, label='COSMOS')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_eg, label='ELAIS')
plt.legend(loc='upper left', labelspacing=0.3, handlelength=1.0, handletextpad=0.2, ncol=2)
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_moonill[cxr]+(0.025*np.random.rand(len(cxr))), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr)
plt.plot( sub_moonill[exr]+(0.025*np.random.rand(len(exr))), sub_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er)
plt.xlabel('Moon Illumination Fraction')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_cr, label='COSMOS')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_er, label='ELAIS')
plt.legend(loc='upper left', labelspacing=0.3, handlelength=1.0, handletextpad=0.2, ncol=2)
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_moonill[cxi]+(0.025*np.random.rand(len(cxi))), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci)
plt.plot( sub_moonill[exi]+(0.025*np.random.rand(len(exi))), sub_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei)
plt.xlabel('Moon Illumination Fraction')
# plt.ylabel('i-band lmt_mg')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_ci, label='COSMOS')
plt.plot( 0,0, 'o', ms=5, alpha=1, color=clr_ei, label='ELAIS')
plt.legend(loc='upper left', labelspacing=0.3, handlelength=1.0, handletextpad=0.2, ncol=2)
plt.ylim([ymin,ymax])
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/imgproc_moonill_vs_limmag')
plt.show()

del ymin,ymax

### zeropoint vs. limiting magnitude

# plt.figure(figsize=(12, 3))
# plt.subplot(1, 3, 1)
# plt.plot( sub_lmg[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_lmg[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('g-band lmt_mg')
# plt.ylabel('g-band mag_zp')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 2)
# plt.plot( sub_lmg[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_lmg[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('r-band lmt_mg')
# # plt.ylabel('r-band mag_zp')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 3)
# plt.plot( sub_lmg[cxi], sub_mzp[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_lmg[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('i-band lmt_mg')
# # plt.ylabel('i-band mag_zp')
# plt.show()

In [ ]:
### moon separation vs. illumination

# plt.figure(figsize=(12, 3))
# plt.rcParams.update({'font.size': 13})
# ax1 = plt.subplot(1, 3, 1)
# plt.plot( sub_moonsep[cxg]+(2.5*np.random.rand(len(cxg))), sub_moonill[cxg]+(0.025*np.random.rand(len(cxg))), 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_moonsep[exg]+(2.5*np.random.rand(len(exg))), sub_moonill[exg]+(0.025*np.random.rand(len(exg))), 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('Moon Separation in Degrees')
# plt.ylabel('Moon Illumination Fraction')
# ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
# plt.plot( sub_moonsep[cxr]+(2.5*np.random.rand(len(cxr))), sub_moonill[cxr]+(0.025*np.random.rand(len(cxr))), 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_moonsep[exr]+(2.5*np.random.rand(len(exr))), sub_moonill[exr]+(0.025*np.random.rand(len(exr))), 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('Moon Separation [deg]')
# ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
# plt.plot( sub_moonsep[cxi]+(2.5*np.random.rand(len(cxi))), sub_moonill[cxi]+(0.025*np.random.rand(len(cxi))), 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_moonsep[exi]+(2.5*np.random.rand(len(exi))), sub_moonill[exi]+(0.025*np.random.rand(len(exi))), 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('Moon Separation [deg]')
# plt.tight_layout(pad=0.7)
# plt.savefig('image_processing_summary_figures/moonsep_vs_moonill')
# plt.show()

<br>
<br>

## Image quality vs. ccd offset from center of image

In [ ]:
# plt.figure(figsize=(12, 3))
# plt.subplot(1, 3, 1)
# plt.plot( sub_off[cxg], sub_rel_lmg[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_off[exg], sub_rel_lmg[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('g-band ccd offset (arcsec)')
# plt.ylabel('g-band sub lmt_mg / exp lmt_mg')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 2)
# plt.plot( sub_off[cxr], sub_rel_lmg[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_off[exr], sub_rel_lmg[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('r-band ccd offset (arcsec)')
# # plt.ylabel('r-band sub lmt_mg / exp lmt_mg')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 3)
# plt.plot( sub_off[cxi], sub_rel_lmg[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_off[exi], sub_rel_lmg[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('i-band ccd offset (arcsec)')
# # plt.ylabel('i-band sub lmt_mg / exp lmt_mg')
# plt.show()

# plt.figure(figsize=(12, 3))
# plt.subplot(1, 3, 1)
# plt.plot( sub_off[cxg], sub_rel_see[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_off[exg], sub_rel_see[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('g-band ccd offset (arcsec)')
# plt.ylabel('g-band sub seeing / exp seeing')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 2)
# plt.plot( sub_off[cxr], sub_rel_see[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_off[exr], sub_rel_see[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('r-band ccd offset (arcsec)')
# # plt.ylabel('r-band sub seeing / exp seeing')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 3)
# plt.plot( sub_off[cxi], sub_rel_see[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_off[exi], sub_rel_see[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('i-band ccd offset (arcsec)')
# # plt.ylabel('i-band sub seeing / exp seeing')
# plt.show()

# plt.figure(figsize=(12, 3))
# plt.subplot(1, 3, 1)
# plt.plot( sub_off[cxg], sub_rel_sks[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_off[exg], sub_rel_sks[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('g-band ccd offset (arcsec)')
# plt.ylabel('g-band sub skysig / exp skysig')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 2)
# plt.plot( sub_off[cxr], sub_rel_sks[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_off[exr], sub_rel_sks[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('r-band ccd offset (arcsec)')
# # plt.ylabel('r-band sub skysig / exp skysig')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 3)
# plt.plot( sub_off[cxi], sub_rel_sks[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_off[exi], sub_rel_sks[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('i-band ccd offset (arcsec)')
# # plt.ylabel('i-band sub skysig / exp skysig')
# plt.show()

# plt.figure(figsize=(12, 3))
# plt.subplot(1, 3, 1)
# plt.plot( sub_off[cxg], sub_rel_mzp[cxg], 'o', ms=2, alpha=0.1, color=clr_cg, label='COSMOS')
# plt.plot( sub_off[exg], sub_rel_mzp[exg], 'o', ms=2, alpha=0.1, color=clr_eg, label='ELAIS')
# plt.xlabel('g-band ccd offset (arcsec)')
# plt.ylabel('g-band sub zeropoint / exp zeropoint')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 2)
# plt.plot( sub_off[cxr], sub_rel_mzp[cxr], 'o', ms=2, alpha=0.1, color=clr_cr, label='COSMOS')
# plt.plot( sub_off[exr], sub_rel_mzp[exr], 'o', ms=2, alpha=0.1, color=clr_er, label='ELAIS')
# plt.xlabel('r-band ccd offset (arcsec)')
# # plt.ylabel('r-band sub zeropoint / exp zeropoint')
# # plt.legend(loc='upper left')
# plt.subplot(1, 3, 3)
# plt.plot( sub_off[cxi], sub_rel_mzp[cxi], 'o', ms=2, alpha=0.1, color=clr_ci, label='COSMOS')
# plt.plot( sub_off[exi], sub_rel_mzp[exi], 'o', ms=2, alpha=0.1, color=clr_ei, label='ELAIS')
# plt.xlabel('i-band ccd offset (arcsec)')
# # plt.ylabel('i-band sub zeropoint / exp zeropoint')
# plt.show()